# Convolutional Neural Networks in ``gluon``

Now let's see how succinctly we can express a convolutional neural network using ``gluon``. You might be relieved to find out that this too requires hardly any more code than a logistic regression. 

In [2]:
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
import numpy as np
mx.random.seed(1)

## Set the context

In [3]:
ctx = mx.gpu()

## Grab the MNIST dataset

In [4]:
mnist = mx.test_utils.get_mnist()
batch_size = 64
train_data = mx.io.NDArrayIter(mnist["train_data"], mnist["train_label"], batch_size, shuffle=True)
test_data = mx.io.NDArrayIter(mnist["test_data"], mnist["test_label"], batch_size, shuffle=True)

## Define a convolutional neural network

Again, a few lines here is all we need in order to change the model. Let's add a couple convolutional layers using ``gluon.nn``.

In [6]:
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Conv2D(channels=20, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))            
    net.add(gluon.nn.Conv2D(channels=50, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))            
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(500, activation="relu"))
    net.add(gluon.nn.Dense(10))

## Parameter initialization


In [7]:
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

## Softmax cross-entropy Loss

In [8]:
loss = gluon.loss.SoftmaxCrossEntropyLoss()

## Optimizer

In [9]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1})

## Write evaluation loop to calculate accuracy

In [10]:
def evaluate_accuracy(data_iterator, net):
    numerator = 0.
    denominator = 0.
    data_iterator.reset()
    for i, batch in enumerate(data_iterator):
        data = batch.data[0].as_in_context(ctx)
        label = batch.label[0].as_in_context(ctx)
        label_one_hot = nd.one_hot(label, 10)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        numerator += nd.sum(predictions == label)
        denominator += data.shape[0]
    return (numerator / denominator).asscalar()

## Training Loop

In [11]:
epochs = 10

for e in range(epochs):
    train_data.reset()
    moving_loss = 0.
    for i, batch in enumerate(train_data):
        data = batch.data[0].as_in_context(ctx)
        label = batch.label[0].as_in_context(ctx)
        with autograd.record():
            output = net(data)
            cross_entropy = loss(output, label)
        cross_entropy.backward()
        trainer.step(data.shape[0])
        
        moving_loss = .99 * moving_loss + .01 * nd.mean(cross_entropy).asscalar()
            
    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % (e, moving_loss, train_accuracy, test_accuracy))    

Epoch 0. Loss: 0.0824594810836, Train_acc 0.974997, Test_acc 0.974721
Epoch 1. Loss: 0.0513552879329, Train_acc 0.985758, Test_acc 0.984674
Epoch 2. Loss: 0.0373197310783, Train_acc 0.991138, Test_acc 0.988356
Epoch 3. Loss: 0.02810394945, Train_acc 0.993587, Test_acc 0.989053
Epoch 4. Loss: 0.0210506014536, Train_acc 0.994869, Test_acc 0.990247
Epoch 5. Loss: 0.016449180497, Train_acc 0.995586, Test_acc 0.989451
Epoch 6. Loss: 0.0124834519165, Train_acc 0.996402, Test_acc 0.990048
Epoch 7. Loss: 0.00941279947156, Train_acc 0.997052, Test_acc 0.990147
Epoch 8. Loss: 0.00775423378892, Train_acc 0.996802, Test_acc 0.990545
Epoch 9. Loss: 0.00613033442457, Train_acc 0.997202, Test_acc 0.991342


## Conclusion

You might notice that by using ``gluon``, we get code that runs much faster whether on CPU or GPU. That's largely because ``gluon`` can call down to highly optimized layers that have been written in C++. 

For whinges or inquiries, [open an issue on  GitHub.](https://github.com/zackchase/mxnet-the-straight-dope)